In [2]:
import pandas as pd

Import all the files

In [4]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [6]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [8]:
visits_cart = pd.merge(visits, 
                       cart,
                       how='left').reset_index()
visits_cart.head()

,index,user_id,visit_time,cart_time
0,0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT
1,1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00
2,2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00
3,3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT
4,4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT


Step 3: How long is `visits_cart`?

In [10]:
total_visits = len(visits_cart)

Step 4: How many timestamps are null for `cart_time`?

In [12]:
null_cart_times = len(visits_cart[visits_cart.cart_time.isnull()])

Step 5: What percentage only visited?

In [37]:
visited_only = round((null_cart_times / total_visits * 100), 2)
visited_only

82.6

Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [45]:
cart_checkout = pd.merge(cart,
                         checkout,
                         how='left').reset_index()
null_checkout = len(cart_checkout[cart_checkout.checkout_time.isnull()])
cart_times = len(cart_checkout)
cart_not_checkout = round((null_checkout / cart_times * 100),2)
cart_not_checkout

35.06

Step 7: Merge it all together

In [87]:
all_data = visits_cart\
.merge(checkout, how = 'left')\
.merge(purchase, how = 'left')

print(all_data.head())

   index                               user_id          visit_time  \
0      0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1      1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2      2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3      3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4      4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3                 NaT                 NaT                 NaT  
4                 NaT                 NaT                 NaT  


Step 8: % of users who got to checkout but did not purchase

In [112]:
reached_checkout = all_data[all_data.checkout_time.notnull()]
no_purchase = all_data[(all_data.purchase_time.isnull()) & (all_data.checkout_time.notnull())]
checkout_no_purchase_percent = round(((len(no_purchase) / len(reached_checkout)) * 100), 2)
checkout_no_purchase_percent

24.55

Step 9: check each part of the funnel, let's print all 3 of them again

In [116]:
print("{} percent of users who visited the page did not add a t-shirt to their cart".format(round(visited_only, 2)))
print("{} percent of users who added a t-shirt to their cart did not checkout".format(round(cart_not_checkout, 2)))
print("{} percent of users who made it to checkout  did not purchase a shirt".format(round( checkout_no_purchase_percent, 2)))

82.6 percent of users who visited the page did not add a t-shirt to their cart
35.06 percent of users who added a t-shirt to their cart did not checkout
24.55 percent of users who made it to checkout  did not purchase a shirt


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [120]:
all_data['time_to_purchase'] = all_data.purchase_time - all_data.visit_time

Step 11: examine the results

In [124]:
all_data.head()

,index,user_id,visit_time,cart_time,checkout_time,purchase_time,time_to_purchase
0,0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT,NaT
1,1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00,0 days 00:44:00
2,2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT,NaT
3,3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT,NaT
4,4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT,NaT


Step 12: average time to purchase

In [126]:
avg_time_to_purchase = all_data.time_to_purchase.mean()
avg_time_to_purchase

Timedelta('0 days 00:43:12.380952380')